In [1]:
import numpy as np
import pandas as pd
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

# Part 1: EDA


## EDA in paymen_report.


In [2]:
payment_report = pd.read_csv('payment_report.csv', encoding = 'utf-8')
payment_report.head()

,report_month,payment_group,product_id,source_id,volume
0,2023-01,payment,12,45,624110375
1,2023-01,payment,17,45,335715113
2,2023-01,payment,18,45,737784466
3,2023-01,payment,19,45,120963069
4,2023-01,payment,20,45,319653158


In [3]:
#Check data type
payment_report.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   report_month   919 non-null    object
 1   payment_group  919 non-null    object
 2   product_id     919 non-null    int64 
 3   source_id      919 non-null    int64 
 4   volume         919 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 36.0+ KB


Incorrect data type 'report_month'  -> Action: convert to data type: datetime \
Incorrect data type 'payment_group' -> Action: convert to data type: string

In [4]:
payment_report['report_month'] = pd.to_datetime(payment_report['report_month'],format = '%Y-%m')
payment_report['payment_group'] = payment_report['payment_group'].astype('string')

In [5]:
payment_report.head()

,report_month,payment_group,product_id,source_id,volume
0,2023-01-01,payment,12,45,624110375
1,2023-01-01,payment,17,45,335715113
2,2023-01-01,payment,18,45,737784466
3,2023-01-01,payment,19,45,120963069
4,2023-01-01,payment,20,45,319653158


In [6]:
payment_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   report_month   919 non-null    datetime64[ns]
 1   payment_group  919 non-null    string        
 2   product_id     919 non-null    int64         
 3   source_id      919 non-null    int64         
 4   volume         919 non-null    int64         
dtypes: datetime64[ns](1), int64(3), string(1)
memory usage: 36.0 KB


In [7]:
#Check missing values in payment_report
payment_report.isnull().sum()

,0
report_month,0
payment_group,0
product_id,0
source_id,0
volume,0


Missing: 0 -> Next step: No Action



In [8]:
payment_report.describe()

,report_month,product_id,source_id,volume
count,919,919.000000,919.000000,9.190000e+02
mean,2023-02-19 06:05:05.549510400,1192.517954,44.875952,1.978574e+08
min,2023-01-01 00:00:00,3.000000,37.000000,5.500000e+03
25%,2023-02-01 00:00:00,640.000000,45.000000,1.250000e+06
50%,2023-03-01 00:00:00,1059.000000,45.000000,7.982015e+06
75%,2023-04-01 00:00:00,1585.000000,45.000000,5.447599e+07
max,2023-04-01 00:00:00,15067.000000,45.000000,1.383171e+10
std,NaN,1293.463329,0.910995,8.367595e+08


In [9]:
#Check duplicate values in payment_report
duplicates_payment = payment_report[payment_report.duplicated(subset=['report_month','payment_group','product_id','source_id'], keep=False)]
duplicates_payment.head()

,report_month,payment_group,product_id,source_id,volume
194,2023-01-01,refund,1976,39,443387276
195,2023-01-01,refund,1976,39,111281678
409,2023-02-01,refund,1976,39,111500
412,2023-02-01,refund,1976,39,910819764
413,2023-02-01,refund,1976,39,2675894726


Check duplicate values in payment_report \\
Primary key: 'report_month','payment_group','product_id','source_id' are duplicated in report month \\
-> The solution is to Groupby the duplicated values ​​together and calculate the total volume value.

In [10]:
#Cleaning duplicate values
cleaning_payment = payment_report.groupby(['report_month','payment_group','product_id','source_id'], as_index=False).agg({'volume': 'sum', 'payment_group': 'first', 'source_id': 'first'})
cleaning_payment.head()

,report_month,product_id,volume,payment_group,source_id
0,2023-01-01,12,624110375,payment,45
1,2023-01-01,17,335715113,payment,45
2,2023-01-01,18,737784466,payment,45
3,2023-01-01,19,120963069,payment,45
4,2023-01-01,20,319653158,payment,45


In [11]:
#Check the value of product_id = 1976 again after cleaning duplicates
cleaning_payment[cleaning_payment['product_id'] == 1976]

,report_month,product_id,volume,payment_group,source_id
192,2023-01-01,1976,852173308,refund,37
193,2023-01-01,1976,3520890417,refund,38
194,2023-01-01,1976,554668954,refund,39
408,2023-02-01,1976,1794436949,refund,37
409,2023-02-01,1976,7522440976,refund,38
410,2023-02-01,1976,3586825990,refund,39
631,2023-03-01,1976,2962187382,refund,37
632,2023-03-01,1976,11652415177,refund,38
633,2023-03-01,1976,6392207104,refund,39
911,2023-04-01,1976,3542271587,refund,37


In [12]:
#Payment_report after EDA
final_payment_report = cleaning_payment[['report_month', 'payment_group', 'product_id', 'source_id', 'volume']]
final_payment_report.head()

,report_month,payment_group,product_id,source_id,volume
0,2023-01-01,payment,12,45,624110375
1,2023-01-01,payment,17,45,335715113
2,2023-01-01,payment,18,45,737784466
3,2023-01-01,payment,19,45,120963069
4,2023-01-01,payment,20,45,319653158


In [13]:
final_payment_report.describe()

,report_month,product_id,source_id,volume
count,914,914.000000,914.000000,9.140000e+02
mean,2023-02-19 07:00:39.387308544,1188.231947,44.908096,1.989397e+08
min,2023-01-01 00:00:00,3.000000,37.000000,5.500000e+03
25%,2023-02-01 00:00:00,640.000000,45.000000,1.246000e+06
50%,2023-03-01 00:00:00,1058.000000,45.000000,7.865532e+06
75%,2023-04-01 00:00:00,1584.000000,45.000000,5.047783e+07
max,2023-04-01 00:00:00,15067.000000,45.000000,1.383171e+10
std,NaN,1295.696596,0.802706,8.612826e+08


## EDA in product


In [14]:
product = pd.read_csv('product.csv',encoding = 'utf-8')
product.head()

,product_id,category,team_own
0,17,PXXXXXB,ASD
1,18,PXXXXXB,ASD
2,20,PXXXXXB,ASD
3,287,PXXXXXB,ASD
4,372,PXXXXXB,ASD


In [15]:
#Check info product
product.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  492 non-null    int64 
 1   category    492 non-null    object
 2   team_own    492 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.7+ KB


Incorrect data type 'Category' -> Action: convert to data type: string \\
Incorrect data type 'Team_own' -> Action: convert to data type: string

In [16]:
product['category'] = product['category'].astype('string')
product['team_own'] = product['team_own'].astype('string')

In [17]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  492 non-null    int64 
 1   category    492 non-null    string
 2   team_own    492 non-null    string
dtypes: int64(1), string(2)
memory usage: 11.7 KB


In [18]:
#Check missing data in product
product.isnull().sum()


,0
product_id,0
category,0
team_own,0


Missing value: 0 -> Next step: No Action

In [19]:
#Check duplicate in product
duplicates_product = product[product.duplicated(subset=['product_id'], keep=False)]
duplicates_product.head()


,product_id,category,team_own


Check duplicate with primary key is product_id -> No duplicate values


## EDA payment_enriched



In [20]:
#Tạo Df payment_enriched bằng merge payment_report & product sau khi làm sạch bảng payment và product
payment_enriched = final_payment_report.merge(product, on='product_id', how = 'left')
payment_enriched.head()

,report_month,payment_group,product_id,source_id,volume,category,team_own
0,2023-01-01,payment,12,45,624110375,PXXXXXT,ASD
1,2023-01-01,payment,17,45,335715113,PXXXXXB,ASD
2,2023-01-01,payment,18,45,737784466,PXXXXXB,ASD
3,2023-01-01,payment,19,45,120963069,PXXXXXM2,ASD
4,2023-01-01,payment,20,45,319653158,PXXXXXB,ASD


In [21]:
payment_enriched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 914 entries, 0 to 913
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   report_month   914 non-null    datetime64[ns]
 1   payment_group  914 non-null    string        
 2   product_id     914 non-null    int64         
 3   source_id      914 non-null    int64         
 4   volume         914 non-null    int64         
 5   category       897 non-null    string        
 6   team_own       897 non-null    string        
dtypes: datetime64[ns](1), int64(3), string(3)
memory usage: 50.1 KB


1. Correct Dtype -> No Action
2. Category và Team_own: missing values -> Check missing values


In [22]:
#Check null value
payment_enriched.isnull().sum()


,0
report_month,0
payment_group,0
product_id,0
source_id,0
volume,0
category,17
team_own,17


In [23]:
#Check for null values
miss_payment_enriched = payment_enriched[payment_enriched['category'].isna() | payment_enriched['team_own'].isna()]
miss_payment_enriched.head()

,report_month,payment_group,product_id,source_id,volume,category,team_own
190,2023-01-01,payment,10033,45,5500,<NA>,<NA>
192,2023-01-01,refund,1976,37,852173308,<NA>,<NA>
193,2023-01-01,refund,1976,38,3520890417,<NA>,<NA>
194,2023-01-01,refund,1976,39,554668954,<NA>,<NA>
405,2023-02-01,payment,10033,45,114050,<NA>,<NA>


In [24]:
cleaned_payment_enriched = payment_enriched.replace(np.nan, 'Unknown')
cleaned_payment_enriched

,report_month,payment_group,product_id,source_id,volume,category,team_own
0,2023-01-01,payment,12,45,624110375,PXXXXXT,ASD
1,2023-01-01,payment,17,45,335715113,PXXXXXB,ASD
2,2023-01-01,payment,18,45,737784466,PXXXXXB,ASD
3,2023-01-01,payment,19,45,120963069,PXXXXXM2,ASD
4,2023-01-01,payment,20,45,319653158,PXXXXXB,ASD
...,...,...,...,...,...,...,...
909,2023-04-01,payment,10039,45,7872489,PXXXXXF,ASL
910,2023-04-01,payment,15067,45,1504000,PXXXXXR,ASL
911,2023-04-01,refund,1976,37,3542271587,Unknown,Unknown
912,2023-04-01,refund,1976,38,13831708189,Unknown,Unknown


Handle Missing values -> Replace < NA > to Unknown

In [25]:
#Check duplicate value
duplicates_payment_enriched = cleaned_payment_enriched[cleaned_payment_enriched.duplicated(subset=['report_month','payment_group','product_id','source_id'], keep=False)]
duplicates_payment_enriched.head()


,report_month,payment_group,product_id,source_id,volume,category,team_own


No duplicate values

In [26]:
#Fianl payment_enriched after EDA
final_payment_enriched = cleaned_payment_enriched[['report_month', 'payment_group', 'product_id', 'source_id', 'volume', 'category', 'team_own']]
final_payment_enriched.head()

,report_month,payment_group,product_id,source_id,volume,category,team_own
0,2023-01-01,payment,12,45,624110375,PXXXXXT,ASD
1,2023-01-01,payment,17,45,335715113,PXXXXXB,ASD
2,2023-01-01,payment,18,45,737784466,PXXXXXB,ASD
3,2023-01-01,payment,19,45,120963069,PXXXXXM2,ASD
4,2023-01-01,payment,20,45,319653158,PXXXXXB,ASD


## EDA transaction


In [27]:
transactions = pd.read_csv('transactions.csv',encoding = 'utf-8')
transactions.head(5)

,transaction_id,merchant_id,volume,transType,transStatus,sender_id,receiver_id,extra_info,timeStamp
0,3002692434,5.0,100000.0,24.0,1.0,10199794.0,199794.0,NaN,1.682932e+12
1,3002692437,305.0,20000.0,2.0,1.0,14022211.0,14022211.0,NaN,1.682932e+12
2,3001960110,7255.0,48605.0,22.0,1.0,NaN,10530940.0,NaN,1.682932e+12
3,3002680710,2270.0,1500000.0,2.0,1.0,10059206.0,59206.0,NaN,1.682932e+12
4,3002680713,2275.0,90000.0,2.0,1.0,10004711.0,4711.0,NaN,1.682932e+12


In [28]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159689 entries, 0 to 159688
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   transaction_id  159689 non-null  int64  
 1   merchant_id     159688 non-null  float64
 2   volume          159688 non-null  float64
 3   transType       159688 non-null  float64
 4   transStatus     159688 non-null  float64
 5   sender_id       153832 non-null  float64
 6   receiver_id     141158 non-null  float64
 7   extra_info      565 non-null     object 
 8   timeStamp       159688 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 11.0+ MB


1. Check Dtype
  - Incorrect data type 'Sender_id' -> Action: convert to int
  - Incorrect data type 'Receiver_id' -> Action: convert to int
  - Incorrect data type 'Extra_info' -> Action: convert to string
2. Check missing values
  - Sender_id: missing values -> Action: replace NaN to 0
  - Receiver_id: missing values -> Action: replace NaN to 0
  - Extra_info: missing values -> Action: ignore



In [29]:
transactions['sender_id'] = transactions['sender_id'].fillna(0).astype('int')
transactions['receiver_id'] = transactions['receiver_id'].fillna(0).astype('int')
transactions['extra_info'] = transactions['extra_info'].astype('string')
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159689 entries, 0 to 159688
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   transaction_id  159689 non-null  int64  
 1   merchant_id     159688 non-null  float64
 2   volume          159688 non-null  float64
 3   transType       159688 non-null  float64
 4   transStatus     159688 non-null  float64
 5   sender_id       159689 non-null  int64  
 6   receiver_id     159689 non-null  int64  
 7   extra_info      565 non-null     string 
 8   timeStamp       159688 non-null  float64
dtypes: float64(5), int64(3), string(1)
memory usage: 11.0 MB


In [30]:
dup_transaction = transactions[transactions.duplicated(subset=['transaction_id'], keep=False)]
dup_transaction.head()

,transaction_id,merchant_id,volume,transType,transStatus,sender_id,receiver_id,extra_info,timeStamp
149734,3000871152,75.0,193000.0,2.0,1.0,17027174,0,<NA>,1.682989e+12
149735,3000871152,75.0,193000.0,2.0,1.0,17027174,0,<NA>,1.682989e+12
149738,3000871161,2270.0,50000.0,2.0,1.0,10009352,9352,<NA>,1.682989e+12
149739,3000871161,2270.0,50000.0,2.0,1.0,10009352,9352,<NA>,1.682989e+12
149752,3000687030,2250.0,165000.0,8.0,1.0,10074844,38548300,<NA>,1.682989e+12


In [31]:
no_dup_transactions = transactions.drop_duplicates(subset = 'transaction_id', ignore_index= False)
no_dup_transactions

,transaction_id,merchant_id,volume,transType,transStatus,sender_id,receiver_id,extra_info,timeStamp
0,3002692434,5.0,100000.0,24.0,1.0,10199794,199794,<NA>,1.682932e+12
1,3002692437,305.0,20000.0,2.0,1.0,14022211,14022211,<NA>,1.682932e+12
2,3001960110,7255.0,48605.0,22.0,1.0,0,10530940,<NA>,1.682932e+12
3,3002680710,2270.0,1500000.0,2.0,1.0,10059206,59206,<NA>,1.682932e+12
4,3002680713,2275.0,90000.0,2.0,1.0,10004711,4711,<NA>,1.682932e+12
...,...,...,...,...,...,...,...,...,...
159684,3001047642,5.0,400.0,22.0,1.0,10028007,15414,<NA>,1.682991e+12
159685,3001019154,2250.0,216.0,8.0,1.0,49000252,127764,<NA>,1.682991e+12
159686,3001047645,7010.0,10000.0,2.0,1.0,10272146,272146,<NA>,1.682991e+12
159687,3001047648,2270.0,40000.0,2.0,1.0,10284844,284844,<NA>,1.682991e+12


Duplicate value with Primary Key: 'transaction_id' -> Action: Drop

# Part 2: Data Wrangling


In [32]:
import numpy as np
import pandas as pd

In [33]:
Data_wrangling = payment_report.merge(product, on='product_id', how = 'left')
Data_wrangling.head()

,report_month,payment_group,product_id,source_id,volume,category,team_own
0,2023-01-01,payment,12,45,624110375,PXXXXXT,ASD
1,2023-01-01,payment,17,45,335715113,PXXXXXB,ASD
2,2023-01-01,payment,18,45,737784466,PXXXXXB,ASD
3,2023-01-01,payment,19,45,120963069,PXXXXXM2,ASD
4,2023-01-01,payment,20,45,319653158,PXXXXXB,ASD


1. Top 3 product_ids with the highest volume


In [34]:
Data_wrangling.groupby('product_id')['volume'].sum().sort_values(ascending=False).head(3)


,volume
product_id,
1976,61797583647
429,14667676567
372,13713658515


2. Given that 1 product_id is only owed by 1 team, are there any abnormal products against this rule?


In [35]:
Data_wrangling['product_team_own'] = (
    Data_wrangling.groupby(['product_id','source_id'])['team_own'].transform('nunique')
)
Data_wrangling.head()

,report_month,payment_group,product_id,source_id,volume,category,team_own,product_team_own
0,2023-01-01,payment,12,45,624110375,PXXXXXT,ASD,1
1,2023-01-01,payment,17,45,335715113,PXXXXXB,ASD,1
2,2023-01-01,payment,18,45,737784466,PXXXXXB,ASD,1
3,2023-01-01,payment,19,45,120963069,PXXXXXM2,ASD,1
4,2023-01-01,payment,20,45,319653158,PXXXXXB,ASD,1


In [36]:
Non_Team = Data_wrangling[Data_wrangling['product_team_own'] != 1].reset_index()
Non_Team.groupby('product_id')['team_own'].nunique()

,team_own
product_id,
3,0
1976,0
10033,0


3. Find the team has had the lowest performance (lowest volume) since Q2.2023. Find the category that contributes the least to that team.


In [37]:
Data_wrangling[Data_wrangling['report_month'] >= '2023-04-01'].groupby(['team_own','category'])['volume'].sum().sort_values(ascending=True).head(1)

,,volume
team_own,category,
ASD,PXXXXXP,300000


4. Find the contribution of source_ids of refund transactions (payment_group = ‘refund’), what is the source_id with the highest contribution?


In [38]:
Data_wrangling[Data_wrangling['payment_group'] == 'refund'].groupby('source_id')['volume'].sum().sort_values(ascending=False).head(1)

,volume
source_id,
38,36527454759


5. Define type of transactions (‘transaction_type’) for each row, given:
  - transType = 2 & merchant_id = 1205: Bank Transfer Transaction
  - transType = 2 & merchant_id = 2260: Withdraw Money Transaction
  - transType = 2 & merchant_id = 2270: Top Up Money Transaction
  - transType = 2 & others merchant_id: Payment Transaction
  - transType = 8, merchant_id = 2250: Transfer Money Transaction
  - transType = 8 & others merchant_id: Split Bill Transaction
  - Remained cases are invalid transactions


In [39]:
transactions = pd.read_csv('transactions.csv')
transactions.head()

,transaction_id,merchant_id,volume,transType,transStatus,sender_id,receiver_id,extra_info,timeStamp
0,3002692434,5,100000.0,24.0,1.0,10199794.0,199794.0,NaN,1.682932e+12
1,3002692437,305,20000.0,2.0,1.0,14022211.0,14022211.0,NaN,1.682932e+12
2,3001960110,7255,48605.0,22.0,1.0,NaN,10530940.0,NaN,1.682932e+12
3,3002680710,2270,1500000.0,2.0,1.0,10059206.0,59206.0,NaN,1.682932e+12
4,3002680713,2275,90000.0,2.0,1.0,10004711.0,4711.0,NaN,1.682932e+12


In [40]:
def transaction_type(row):
    if row['transType'] == 2 and row['merchant_id'] == 1205:
        return 'Bank Transfer Transaction'
    elif row['transType'] == 2 and row['merchant_id'] == 2260:
        return 'Withdraw Money Transaction'
    elif row['transType'] == 2 and row['merchant_id'] == 2270:
        return 'Top Up Money Transaction'
    elif row['transType'] == 2 and row['merchant_id'] not in [1205,2260,2270]:
        return 'Payment Transaction'
    elif row['transType'] == 8 and row['merchant_id'] == 2250:
        return 'Transfer Money Transaction'
    elif row['transType'] == 8:
        return 'Split Bill Transaction'
    else:
        return 'Invalid Transaction'


In [41]:
transactions['Type'] = transactions.apply(transaction_type, axis=1)
transactions.head()

,transaction_id,merchant_id,volume,transType,transStatus,sender_id,receiver_id,extra_info,timeStamp,Type
0,3002692434,5,100000.0,24.0,1.0,10199794.0,199794.0,NaN,1.682932e+12,Invalid Transaction
1,3002692437,305,20000.0,2.0,1.0,14022211.0,14022211.0,NaN,1.682932e+12,Payment Transaction
2,3001960110,7255,48605.0,22.0,1.0,NaN,10530940.0,NaN,1.682932e+12,Invalid Transaction
3,3002680710,2270,1500000.0,2.0,1.0,10059206.0,59206.0,NaN,1.682932e+12,Top Up Money Transaction
4,3002680713,2275,90000.0,2.0,1.0,10004711.0,4711.0,NaN,1.682932e+12,Payment Transaction


6. Of each transaction type (excluding invalid transactions): find the number of transactions, volume, senders and receivers.

In [42]:
transactions[transactions['Type'] != 'Invalid Transaction'].groupby('Type').agg({'transaction_id': 'count'
                                                                                , 'volume': 'sum'
                                                                                , 'sender_id': 'nunique'
                                                                                , 'receiver_id': 'nunique'})

,transaction_id,volume,sender_id,receiver_id
Type,,,,
Bank Transfer Transaction,5185,7.440799e+09,4338,1743
Payment Transaction,52622,8.667976e+09,37092,27020
Split Bill Transaction,171,6.398060e+05,166,110
Top Up Money Transaction,39284,1.422743e+10,28411,28411
Transfer Money Transaction,46710,5.098194e+09,11853,8212
Withdraw Money Transaction,4327,2.766309e+09,4051,4051
